<a href="https://colab.research.google.com/github/eisbetterthanpi/JEPA/blob/main/guitar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install perceiver-io[text,vision,audio]
# !pip install perceiver-io[audio]
!pip install perceiver-io audio

# !apt update && apt install -y fluidsynth


In [ ]:

# https://github.com/krasserm/perceiver-io
from transformers import pipeline
from pretty_midi import PrettyMIDI
from perceiver.model.audio import symbolic  # auto-class registration

# https://huggingface.co/krasserm/perceiver-ar-sam-giant-midi
repo_id = "krasserm/perceiver-ar-sam-giant-midi"
audio_generator = pipeline("symbolic-audio-generation", model=repo_id)

prompt = PrettyMIDI("prompt.mid")
output = audio_generator(prompt, max_new_tokens=64, num_latents=1, do_sample=True, top_p=0.95, temperature=1.0, render=True)

with open("generated_audio.wav", "wb") as f:
    f.write(output["generated_audio_wav"])


In [ ]:
import IPython
from perceiver.model.audio import symbolic  # auto-class registration
from pretty_midi import PrettyMIDI
# https://colab.research.google.com/github/krasserm/perceiver-io/blob/main/examples/inference.ipynb#scrollTo=AZAN_Jq4UzKN


# 134M parameters and was trained with a context of 4096 prefix tokens and 2048 latent tokens for 27 epochs (157M tokens per epoch) on the GiantMIDI-Piano dataset

# tokenization method in Section A.2 of # music transformer, jbchorals # https://arxiv.org/pdf/1809.04281
# midi processor from # https://github.com/jason9693/midi-neural-processor/blob/master/processor.py


# midi prompt
res = requests.get("https://martin-krasser.com/perceiver/data/midi/prompt.mid")
with open("prompt.mid", "wb") as f:
    f.write(res.content)

# soundfont used to render the generated MIDI symbols into WAV format
res = requests.get("https://martin-krasser.com/perceiver/data/midi/SGM-v2.01-NicePianosGuitarsBass-V1.2.sf2")
with open("SGM-v2.01-NicePianosGuitarsBass-V1.2.sf2", "wb") as f:
    f.write(res.content)


In [ ]:

audio_generator = pipeline("symbolic-audio-generation", model="krasserm/perceiver-ar-sam-giant-midi", device="cuda:0")

prompt = PrettyMIDI("prompt.mid")
# generated sequence starts at timestamp 14s
output = audio_generator(prompt, max_new_tokens=1024, max_prompt_length=256, num_latents=1, do_sample=True,
    top_p=0.95, temperature=1.0, return_full_audio=True, render=True, sf2_path="SGM-v2.01-NicePianosGuitarsBass-V1.2.sf2")

output = audio_generator(prompt, max_new_tokens=1024, max_prompt_length=4096, num_latents=1, do_sample=True,
    top_p=0.95, temperature=1.0, return_full_audio=False, render=True, sf2_path="SGM-v2.01-NicePianosGuitarsBass-V1.2.sf2")
# return_full_audio=True: generated sequence is appended to the prompt
# The initial number of latent positions is set to `1` corresponding to the last `1` token of the prompt sequence

with open("out.wav", "wb") as f:
    f.write(output["generated_audio_wav"])

IPython.display.Audio("out.wav")


In [ ]:
# @title training


# https://github.com/krasserm/perceiver-io/blob/main/docs/training-examples.md#symbolic-audio-modeling






In [ ]:

# https://huggingface.co/datasets/juancopi81/mutopia_guitar_dataset?row=0


In [ ]:
# @title guitarset
import requests
url = 'https://zenodo.org/records/3371780/files/annotation.zip?download=1'
with open('annotation.zip', 'wb') as f:
    f.write(requests.get(url).content)

!unzip annotation.zip -d guitarset
!pip install jams pretty_midi


In [ ]:
# @title jams_to_midi
import jams
import pretty_midi
import numpy as np

# https://github.com/marl/GuitarSet/blob/master/visualize/interpreter.py
def jams_to_midi(jam, q=1):
    # q = 1: with pitch bend. q = 0: without pitch bend.
    midi = pretty_midi.PrettyMIDI()
    annos = jam.search(namespace='note_midi')
    if len(annos) == 0:
        annos = jam.search(namespace='pitch_midi')
    for anno in annos:
        midi_ch = pretty_midi.Instrument(program=25) # https://en.wikipedia.org/wiki/General_MIDI
        for note in anno:
            # print(note)
            pitch = int(round(note.value))
            bend_amount = int(round((note.value - pitch) * 4096))
            st = note.time
            dur = note.duration
            n = pretty_midi.Note(velocity=100 + np.random.choice(range(-5, 5)), pitch=pitch, start=st, end=st+dur)
            # n = pretty_midi.Note(velocity= , pitch=pitch, start=st, end=st+dur)
            pb = pretty_midi.PitchBend(pitch=bend_amount * q, time=st)
            midi_ch.notes.append(n)
            midi_ch.pitch_bends.append(pb)
        if len(midi_ch.notes) != 0:
            midi.instruments.append(midi_ch)
    return midi

import os
directory = "guitarset"
for i, filename in enumerate(os.listdir(directory)):
    f = os.path.join(directory, filename)
    print(f)
    jam = jams.load(f)
    midi = jams_to_midi(jam)
    midi.write('midi{i}.mid'.format(i=i))
    if i>4:
        break


In [ ]:
url = 'https://www.theguitarlesson.com/guitar-pro-tabs/category/a/a-perfect-circle/page/5/'
out = requests.get(url).content
print(out)


In [ ]:
# @title from selenium_house.ipynb
from urllib.parse import unquote
import re
def get_imgurls(driver):
    pro=driver.page_source
    # pro=driver.execute_script("return document.documentElement.outerHTML")
    pro = unquote(pro)
    pro = pro.encode().decode('unicode-escape') # try to remove \\u00
    # pattern = r"https://(?:(?!\").)*\.jpg" # start with https:// , no " , end with .jpg # https://stackoverflow.com/a/67540659/13359815
    pattern = r"(?<=href=\"/imgres\?imgurl=)(?:(?!&amp;).)*(?=&amp;)" # start with href="/imgres?imgurl= , no &amp; , end with &amp; # https://stackoverflow.com/a/3926546/13359815
    # pattern = r"https://(?:(?!\").)*(?=\")" # start with https:// , no " , end with " # all urls
    # pattern = r"(?<=\],\[\")https://(?:(?!\").)*(?=\")" # start with ],[" then https:// , no " , end with " # all imgs? fast
    m = re.findall(pattern, pro)
    return m


from concurrent.futures import ThreadPoolExecutor
import time
for i in range(0, len(m), batch_size):
    # i=0
    print(i)
    start = time.time()
    # e = ThreadPoolExecutor(batch_size) # 19.788sec
    e = ThreadPoolExecutor(min(batch_size, len(m))) # 21sec

    # result = e.map(get_img, m[i*batch_size: min((i+1)*batch_size, len(m))])
    # # result = e.map(get_img, m)
    # result = list(filter(lambda item: item is not None, result))

    # future = [e.submit(get_img, uuu) for uuu in m[i*batch_size: min((i+1)*batch_size, len(m))]]
    future = [e.submit(get_img, uuu) for uuu in m[i: min(i+batch_size, len(m))]]
    # result = [f.result() for f in future if f.result() is not None]
    ulst = [f.result()[0] for f in future if f.result() is not None]
    images = [f.result()[1] for f in future if f.result() is not None]
    # imgidx = [i for i in range(len(future)) if future[i].result() is not None] # only use for indexing m
    # print(len(ulst))

    end = time.time()
    print(end-start)
    start=end


In [ ]:
# @title all
# url = 'https://www.theguitarlesson.com/guitar-pro-tabs/a/a-perfect-circle/judith-a-perfect-circle/'
# url = 'https://www.theguitarlesson.com/guitar-pro-tabs/c/connells-the/74-75-2-connells-the/'
# url = 'https://www.theguitarlesson.com/guitar-pro-tabs/c/connells-the/74-75-connells-the/'
url = 'https://www.theguitarlesson.com/guitar-pro-tabs/0-9/31-minutos/tema-central-31-minutos/'
out = requests.get(url).content
print(out)
# File Name:</strong> <a href="https://www.theguitarlesson.com/guitar-pro-tabs/song-files/A Perfect Circle - Judith.gp4"

from urllib.parse import unquote
import re
pro=out
pro = unquote(pro)
# pattern = r"(?<=File Name:</strong> <a href=\"https://www.theguitarlesson.com/guitar-pro-tabs/song-files/)(?:(?!\").)*(?=\")"
pattern = r"(?<=<a href=\"https://www.theguitarlesson.com/guitar-pro-tabs/song-files/)(?:(?!\").)*(?=\")"
m = re.findall(pattern, pro)
print(m)
if len(m)>1: m=max(m)
else: m=m[0]
gp_url = "https://www.theguitarlesson.com/guitar-pro-tabs/song-files/"+m
print(gp_url)

name,gpnum = m.split(' - ')[-1].split('.')
name=name.replace(' ', '_')
print(name)
print(gpnum)

# gp_url = 'https://www.theguitarlesson.com/guitar-pro-tabs/song-files/A Perfect Circle - Judith.gp4'
# Connells (The) – 74-75.gp3
# Connells (The) – 74-75 (2).gp3

res = requests.get(gp_url)
# print(out)
with open(name+'.'+gpnum, "wb") as f:
    f.write(res.content)

!/content/GuitarProToMidi /content/{name}.{gpnum} -o /content/midi/{name}.mid


b'<!doctype html>\n<html lang="en-US" class="no-js" itemtype="https://schema.org/Blog" itemscope>\n<head>\n\t<meta charset="UTF-8">\n\t<meta name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1">\n\t<meta name=\'robots\' content=\'index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1\' />\n \n    <!-- Global site tag (gtag.js) - Google Analytics -->\n    <script async src="https://www.googletagmanager.com/gtag/js?id=UA-5257838-7"></script>\n    <script>\n      window.dataLayer = window.dataLayer || [];\n      function gtag(){dataLayer.push(arguments);}\n      gtag(\'js\', new Date());\n\n      gtag(\'config\', \'UA-5257838-7\');\n    </script> \n\n\t<!-- This site is optimized with the Yoast SEO plugin v22.2 - https://yoast.com/wordpress/plugins/seo/ -->\n\t<title>Tema Central - 31 Minutos - Guitar Pro Tabs</title>\n\t<link rel="canonical" href="https://www.theguitarlesson.com/guitar-pro-tabs/0-9/31-minutos/tema-central-31-minutos/" />\n\

In [1]:
import os
os.mkdir("/content/midi")
os.mkdir("/content/gp")

In [2]:
# @title setup GuitarProToMidi
# https://github.com/rageagainsthepc/GuitarPro-to-Midi
import requests

url = 'https://github.com/rageagainsthepc/GuitarPro-to-Midi/releases/download/v0.1.6/guitarprotomidi_linux-x64.zip'
res = requests.get(url)
# print(out)
with open("guitarprotomidi_linux.zip", "wb") as f:
    f.write(res.content)
!unzip guitarprotomidi_linux.zip

# import sys
# # sys.path.append('/content/OpenVoice/openvoice')
# sys.path.append('/content/GuitarProToMidi')
# sys.path.append('/content')

!chmod 755 /content/GuitarProToMidi


# !/content/GuitarProToMidi /content/Judith.gp4
# !/content/GuitarProToMidi /content/Judith.gp4 -o /content/out.mid

# !/content/GuitarProToMidi /content/{Judith}.gp4 -o /content/midi/{out}.mid
# !/content/GuitarProToMidi -h


Archive:  guitarprotomidi_linux.zip
  inflating: GuitarProToMidi         


In [ ]:
# @title as func
import requests
from urllib.parse import unquote
import re

def run_cat(c):
    cat_url = 'https://www.theguitarlesson.com/guitar-pro-tabs/'+c
    if c=='0-9-songs': c='0-9'
    out = requests.get(cat_url).content
    pro = unquote(out)
    # print(cat_url)
    pattern = r"(?<=<a href=\"https://www.theguitarlesson.com/guitar-pro-tabs/category/{}/)(?:(?!\").)*(?=\")".format(c)
    m = re.findall(pattern, pro)
    # print(m)
    art_urls = ['https://www.theguitarlesson.com/guitar-pro-tabs/category/'+c+'/'+mm for mm in m]
    return art_urls


# url = 'https://www.theguitarlesson.com/guitar-pro-tabs/category/a/a-perfect-circle/'

def get_song_urls(url):
    # print('get_song_urls',url)
    out = requests.get(url).content
    pro = unquote(out)
    pattern = r"(?<=>)(?:(?!<).)*(?=</a>\n<a class=\"next page-numbers\")".format(c)
    m = re.findall(pattern, pro)
    # print('get_song_urls',m)
    if len(m)==0:
        out = requests.get(url).content
        pro = unquote(out)
        pattern = r"(?<=class=\"entry-title\"><a href=\"https://www.theguitarlesson.com/guitar-pro-tabs/)(?:(?!\").)*(?=\")".format(c)
        song_urls = re.findall(pattern, pro)
    else:
        num_pages=int(m[0])
        song_urls=[]
        for pg in range(1,num_pages+1):
            pg_url = url+'page/'+str(pg)
            out = requests.get(pg_url).content
            pro = unquote(out)
            pattern = r"(?<=class=\"entry-title\"><a href=\"https://www.theguitarlesson.com/guitar-pro-tabs/)(?:(?!\").)*(?=\")".format(c)
            m = re.findall(pattern, pro)
            song_urls+=m
    return song_urls

# url = 'https://www.theguitarlesson.com/guitar-pro-tabs/'
# print([url+x for x in song_urls])

# url = 'https://www.theguitarlesson.com/guitar-pro-tabs/c/connells-the/74-75-connells-the/'

def clean_name(m):
    o=m.find('(')
    if o>0:
        p=m.find(')')
        brac=m[o+1:p]
        m=brac+' '+m[:o-1]+m[p+1:]
    m=re.sub(r'[^\w]', ' ', m)
    m=m.replace(',', '').replace('.', '').replace(' - ','-').replace(' – ','-').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace(' ', '_')
    if m[-1]=='_': m=m[:-1]
    return m

def clean_all(m):
    o=m.find(' – ')
    if o>0: artist,tgp=m.split(' – ')
    else:
        o=m.find(' - ')
        if o>0: artist,tgp=m.split(' - ')
    title,gpnum = tgp[:-4],tgp[-3:]
    title = re.sub(r'[^\w]', ' ', title)
    title=title.replace(',', '').replace('.', '').replace(' - ','-').replace(' – ','-').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace(' ', '_')
    if title[-1]=='_': title=title[:-1]
    artist = clean_name(artist)
    return title,artist,gpnum

# title,artist,gpnum = clean_all(m)

def url_to_midi(url):
    pro = requests.get(url).content
    pro = unquote(pro)
    # pattern = r"(?<=File Name:</strong> <a href=\"https://www.theguitarlesson.com/guitar-pro-tabs/song-files/)(?:(?!\").)*(?=\")"
    pattern = r"(?<=<a href=\"https://www.theguitarlesson.com/guitar-pro-tabs/song-files/)(?:(?!\").)*(?=\")"
    m = re.findall(pattern, pro)
    try:
        if len(m)>1: m=max(m)
        else: m=m[0]
        title,artist,gpnum = clean_all(m)
        name = title+'-'+artist
        # name,gpnum = clean_name(m)
        gp_url = "https://www.theguitarlesson.com/guitar-pro-tabs/song-files/"+m
        res = requests.get(gp_url)
        with open('/content/gp/'+name+'.'+gpnum, "wb") as f:
            f.write(res.content)
        !/content/GuitarProToMidi /content/gp/{name}.{gpnum} -o /content/midi/{name}.mid
    except:
        print('url_to_midi error', url)
        pass

from concurrent.futures import ThreadPoolExecutor
cat = ['0-9-songs'] + [chr(i) for i in range(97,123)] # 97 a # 122 z
id=2*0
for c in cat[:2]:
    art_urls = run_cat(c)
    for art_url in art_urls:
        song_urls = get_song_urls(art_url)
        # print('song_urls',song_urls)
        song_urls2 = ['https://www.theguitarlesson.com/guitar-pro-tabs/'+x for x in song_urls]
        e = ThreadPoolExecutor(1024) # 21sec
        for song_url in song_urls2:
            e.submit(url_to_midi, song_url)
            # url_to_midi(song_url)



=Error: Index was out of range. Must be non-negative and less than the size of the collection. (Parameter 'index')
=Error: Unhandled value. (Parameter 'harmonicFret')
Actual value was 0.
=Error: The file '/content/midi/Money_Money_Money-Abba.mid' already exists.


In [ ]:
%cd /content/midi
!zip -r /content/midi.zip /content/midi


In [ ]:
# @title clean
m=' Connells ` Jr. (The) – 74-75` (2) .gp3'
# m='1349 – Deathmarch (2).gp4'
# name,gpnum = m.split('.')
def clean_name(m):
    o=m.find('(')
    if o>0:
        p=m.find(')')
        brac=m[o+1:p]
        m=brac+' '+m[:o-1]+m[p+1:]
    m=re.sub(r'[^\w]', ' ', m)
    m=m.replace(',', '').replace('.', '').replace(' - ','-').replace(' – ','-').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace(' ', '_')
    if m[-1]=='_': m=m[:-1]
    return m

def clean_all(m):
    o=m.find(' – ')
    if o>0: artist,tgp=m.split(' – ')
    else:
        o=m.find(' - ')
        if o>0: artist,tgp=m.split(' - ')
    title,gpnum = tgp[:-4],tgp[-3:]
    title = re.sub(r'[^\w]', ' ', title)
    title=title.replace(',', '').replace('.', '').replace(' - ','-').replace(' – ','-').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace(' ', '_')
    if title[-1]=='_': title=title[:-1]
    artist = clean_name(artist)
    return title,artist,gpnum


title,artist,gpnum = clean_all(m)
print(title)
print(artist)
print(gpnum)
name = title+'-'+artist
print(name)

74_75_2
The_Connells_Jr
gp3
74_75_2-The_Connells_Jr


In [ ]:
# @title categories
# cat = ['0-9-songs'] + [chr(i) for i in range(97,123)] # 97 a # 122 z
# print(cat)

# for c in cat:
#     cat_url = 'https://www.theguitarlesson.com/guitar-pro-tabs/'+c
#     break

cat_url = 'https://www.theguitarlesson.com/guitar-pro-tabs/0-9-songs'
# cat_url = 'https://www.theguitarlesson.com/guitar-pro-tabs/r'
out = requests.get(cat_url).content
# print(out)
# <a href="https://www.theguitarlesson.com/guitar-pro-tabs/category/r/rem/">R.E.M</a> (39)<br />\n\t

# c='r'
c='0-9-songs'
if c=='0-9-songs': c='0-9'
pro = unquote(out)
pattern = r"(?<=<a href=\"https://www.theguitarlesson.com/guitar-pro-tabs/category/{}/)(?:(?!\").)*(?=\")".format(c)
m = re.findall(pattern, pro)
print(m)
cat_url
# https://www.theguitarlesson.com/guitar-pro-tabs/category/r/rem/

# art_urls=[]

def run_cat(c):
    cat_url = 'https://www.theguitarlesson.com/guitar-pro-tabs/'+c
    if c=='0-9-songs': c='0-9'
    out = requests.get(cat_url).content
    pro = unquote(out)
    # print(cat_url)
    pattern = r"(?<=<a href=\"https://www.theguitarlesson.com/guitar-pro-tabs/category/{}/)(?:(?!\").)*(?=\")".format(c)
    m = re.findall(pattern, pro)
    # print(m)
    art_urls = ['https://www.theguitarlesson.com/guitar-pro-tabs/category/'+c+'/'+mm for mm in m]
    return art_urls
# https://www.theguitarlesson.com/guitar-pro-tabs/category/0-9/1349/
c='0-9-songs'
art_urls = run_cat(c)
print(art_urls)


['1349/', '311/', '1-the/', '1-god/', '10-years/', '12-stones/', '1974-ad-pahilo-junema/', '1kempfi1/', '1st-man-for-her/', '2-unlimited/', '20-70/', '23-ph/', '28-days/', '2pac/', '2wo/', '3-doors-down/', '3-inches-of-blood/', '30-seconds-to-mars/', '301-izquierda/', '31-minutos/', '36-crazyfists/', '38-special/', '3rd-strike/', '4-non-blondes/', '4-skins-the/', '4d-sports-driving/', '4lyn/', '5-6-7-8s-the/', '50-cent/', '500-and-china-blue/', '59-times-the-pain/', '5nizza/', '5th-prokurator/', '6-voltios/', '69-eyes/', '69ers-the/', '704-exercises-for-guitar-1/', '7b/', '8-foot-sativa/', '88-fingers-louie/', '8forty8/', '8stops7/']
['https://www.theguitarlesson.com/guitar-pro-tabs/category/0-9/1349/', 'https://www.theguitarlesson.com/guitar-pro-tabs/category/0-9/311/', 'https://www.theguitarlesson.com/guitar-pro-tabs/category/0-9/1-the/', 'https://www.theguitarlesson.com/guitar-pro-tabs/category/0-9/1-god/', 'https://www.theguitarlesson.com/guitar-pro-tabs/category/0-9/10-years/', 'h

In [ ]:
# @title song urls
# https://www.theguitarlesson.com/guitar-pro-tabs/category/a/a-perfect-circle/
# url = 'https://www.theguitarlesson.com/guitar-pro-tabs/category/a/a-perfect-circle'
# url = 'https://www.theguitarlesson.com/guitar-pro-tabs/category/a/a-perfect-circle/'
url='https://www.theguitarlesson.com/guitar-pro-tabs/category/0-9/1349/'
# out = requests.get(url).content
# print(out)
# <!-- .entry-taxonomies -->\n\t\t<h2 class="entry-title"><a href="https://www.theguitarlesson.com/guitar-pro-tabs/a/a-perfect-circle/judith-a-perfect-circle/" rel="bookmark">Judith &#8211; A Perfect Circle</a></h2></header><!-- .entry-header -->\n<footer class="entry-footer">\n\

# <p class="infinite-scroll-last">End of content</p><p class="infinite-scroll-error">End of content</p></div>\n\t<nav class="navigation pagination" aria-label="Page navigation">\n\t\t<h2 class="screen-reader-text">Page navigation</h2>\n\t\t<div class="nav-links"><span aria-current="page" class="page-numbers current">1</span>\n<a class="page-numbers" href="https://www.theguitarlesson.com/guitar-pro-tabs/category/a/a-perfect-circle/page/2/">2</a>\n<a class="page-numbers" href="https://www.theguitarlesson.com/guitar-pro-tabs/category/a/a-perfect-circle/page/3/">3</a>\n<span class="page-numbers dots">


# class="entry-title"><a href="https://www.theguitarlesson.com/guitar-pro-tabs

pro = unquote(out)
pattern = r"(?<=>)(?:(?!<).)*(?=</a>\n<a class=\"next page-numbers\")".format(c)
m = re.findall(pattern, pro)
# > </a>\n<a class="next page-numbers"
print(m)

if len(m)==0:
    out = requests.get(url).content
    pro = unquote(out)
    pattern = r"(?<=class=\"entry-title\"><a href=\"https://www.theguitarlesson.com/guitar-pro-tabs/)(?:(?!\").)*(?=\")".format(c)
    return re.findall(pattern, pro)
else:
    num_pages=int(m[0])
    song_urls=[]
    for pg in range(1,num_pages+1):
        pg_url = url+'page/'+str(pg)
        out = requests.get(pg_url).content
        pro = unquote(out)
        pattern = r"(?<=class=\"entry-title\"><a href=\"https://www.theguitarlesson.com/guitar-pro-tabs/)(?:(?!\").)*(?=\")".format(c)
        m = re.findall(pattern, pro)
        song_urls+=m
return song_urls

song_urls=[]
for pg in range(1,num_pages+1):
    pg_url = 'https://www.theguitarlesson.com/guitar-pro-tabs/category/a/a-perfect-circle/'+'page/'+str(pg)
    # print(pg_url)
    out = requests.get(pg_url).content
    # print(out)
    pro = unquote(out)
    pattern = r"(?<=class=\"entry-title\"><a href=\"https://www.theguitarlesson.com/guitar-pro-tabs/)(?:(?!\").)*(?=\")".format(c)
    m = re.findall(pattern, pro)
    print(m)
    song_urls+=m
print(song_urls)
print(len(song_urls))
# 46


# for a_url in art_urls:
url = 'https://www.theguitarlesson.com/guitar-pro-tabs/'
print([url+x for x in song_urls])


In [ ]:
from pretty_midi import PrettyMIDI

prompt = PrettyMIDI("/content/midi/74-75.mid")
# print(prompt)

# print(prompt.instruments)
# print(prompt.instruments[1].notes)
print(prompt.instruments[1].pitch_bends)
# print(prompt.notes)
# prettymidi > instruments > notes(start, end, pitch, velocity) + pitch_bends


[]


In [ ]:
# @title merge_midi

def merge_midi(jam, q=1):
    # q = 1: with pitch bend. q = 0: without pitch bend.
    midi = pretty_midi.PrettyMIDI()
    annos = jam.search(namespace='note_midi')
    if len(annos) == 0:
        annos = jam.search(namespace='pitch_midi')
    for anno in annos:
        midi_ch = pretty_midi.Instrument(program=25) # https://en.wikipedia.org/wiki/General_MIDI
        for note in anno:
            # print(note)
            pitch = int(round(note.value))
            bend_amount = int(round((note.value - pitch) * 4096))
            st = note.time
            dur = note.duration
            n = pretty_midi.Note(velocity=100 + np.random.choice(range(-5, 5)), pitch=pitch, start=st, end=st+dur)
            # n = pretty_midi.Note(velocity= , pitch=pitch, start=st, end=st+dur)
            pb = pretty_midi.PitchBend(pitch=bend_amount * q, time=st)
            midi_ch.notes.append(n)
            midi_ch.pitch_bends.append(pb)
        if len(midi_ch.notes) != 0:
            midi.instruments.append(midi_ch)
    return midi

# prompt.instruments[0].notes.append(prompt.instruments[1].notes)



In [ ]:

# https://www.guitar-pro.com/tabs/genres
# https://www.guitar-pro.com/tabs/genres/2-pop


